# Tensor Network

In [1]:
using TenetCore

The most fundamental interface in TenetCore is `TensorNetwork`. It defines the methods that need to be implemented by a "Tensor Network": access to tensors, inds and adding/removing/replacing them too. It is defined in [src/Interfaces/TensorNetwork.jl](../src/Interfaces/TensorNetwork.jl).

`SimpleTensorNetwork` is a concrete type that implements both the `TensorNetwork` and `Network` interfaces , but in general.

> [!NOTE]
> `SimpleTensorNetwork` is basically a copy-paste of Tenet's `TensorNetwork` refactored to fit the new interfaces.

In [2]:
A = Tensor(rand(2,2), [Index(:i), Index(:j)])
B = Tensor(rand(2,2), [Index(:j), Index(:k)])
tn = SimpleTensorNetwork([A, B])

SimpleTensorNetwork (#tensors=2, #inds=3)

## Tensors

To access the tensors in a Tensor Network, there is the explicit `tensors_*` methods or the keyword-dispatched `tensors` function.

If you want to get all the tensors, you can call `all_tensors` or just `tensors`.

In [3]:
tensors(tn)

2-element Vector{Tensor}:
 [0.29942422960966175 0.9426442933561636; 0.44413244363982096 0.8089520718065251]
 [0.5730113605575105 0.017342601634127597; 0.5408191116093481 0.13063706837810418]

`tensors_with_inds` (aka `tensors(; withinds)`) returns the `Tensor`s whose `inds` match the ones passed down.

In [4]:
tensors(tn; withinds=[Index(:i), Index(:j)])

1-element Vector{Tensor{Float64, 2, Matrix{Float64}}}:
 [0.5730113605575105 0.017342601634127597; 0.5408191116093481 0.13063706837810418]

`tensors_contain_inds` (aka `tensors(; contain)`) returns all the `Tensor`s whose `inds` contain **all** the indices you pass.

In [5]:
tensors(tn; contain=Index(:i))

1-element Vector{Tensor{Float64, 2, Matrix{Float64}}}:
 [0.5730113605575105 0.017342601634127597; 0.5408191116093481 0.13063706837810418]

`tensors_intersect_inds` (aka `tensors(; intersect)`) returns all the `Tensors` whose `inds` **intersect** with the ones you pass (i.e. contain at least one).

In [6]:
tensors(tn; intersect=Index(:i))

1-element Vector{Tensor}:
 [0.5730113605575105 0.017342601634127597; 0.5408191116093481 0.13063706837810418]

The difference between `intersect` and `contain` arises when you pass a list of indices.

In [7]:
tensors(tn; contain=[Index(:i), Index(:k)])

Tensor{Float64, 2, Matrix{Float64}}[]

In [8]:
tensors(tn; intersect=[Index(:i), Index(:k)])

2-element Vector{Tensor}:
 [0.29942422960966175 0.9426442933561636; 0.44413244363982096 0.8089520718065251]
 [0.5730113605575105 0.017342601634127597; 0.5408191116093481 0.13063706837810418]

`ntensors` returns the number of tensors in a Tensor Network, and accepts the same keyword arguments as `tensors`. The reason for it to be used instead of `length(tensors(...))` is that it can be wayyy faster on some ocassions, so it's better to use `ntensors` if you're gonna call it a lot of times.

In [9]:
ntensors(tn)

2

In [10]:
ntensors(tn; contain=Index(:j))

2

`hastensor` checks if a certain `Tensor` is inside the Tensor Network. Note that in TenetCore (and Tenet), we use "egality" (i.e. `===`) to check for belonging. So even if 2 `Tensor`s are equal, it won't matter: it needs to be the **exact same tensor**.

In [11]:
hastensor(tn, A)

true

In [12]:
hastensor(tn, Tensor(rand(2,2), [Index(:i), Index(:j)]))

false

## Indices

`all_inds` (aka `inds`) returns all the indices present in the Tensor Network.

In [13]:
inds(tn)

3-element Vector{Index}:
 Index{Symbol}(:j)
 Index{Symbol}(:k)
 Index{Symbol}(:i)

`inds_set_open` (aka `inds(; set=:open)`) returns the open indices (i.e. indices appearing in only 1 tensor).

In [14]:
inds(tn; set=:open)

2-element Vector{Index}:
 Index{Symbol}(:i)
 Index{Symbol}(:k)

`inds_set_inner` (aka `inds(; set=:inner)`) returns the inner indices (i.e. indices appearing in just 2 tensors).

In [15]:
inds(tn; set=:inner)

1-element Vector{Index}:
 Index{Symbol}(:j)

`inds_set_hyper` (aka `inds(; set=:hyper)`) returns the hyper indices (i.e. indices appearing in 3 or more tensors).

In [16]:
inds(tn; set=:hyper)

Index[]

`inds_parallel_to` (aka `inds(; parallel_to)`) returns any other index "parallel" to the one passed.

In [17]:
inds(tn; parallel_to=Index(:i))

Index[]

Just like with `ntensors` and `hastensor`, there exist its `inds` counterparts: `ninds` and `hasind`.

In [18]:
ninds(tn)

3

In [19]:
ninds(tn; set=:open)

2

In [20]:
hasind(tn, Index(:i))

true

In [21]:
hasind(tn, Index(:not_i))

false

Calling `size_inds` (aka `Base.size`) on a Tensor Network, returns a `Dict` that maps `Index` to their sizes.

In [22]:
size(tn)

Dict{Index, Int64} with 3 entries:
  Index{Symbol}(:i) => 2
  Index{Symbol}(:j) => 2
  Index{Symbol}(:k) => 2

`size_ind` (aka `Base.size` with an `Index` as the 2nd argument) returns the size for just the passed `Index`.

In [23]:
size(tn, Index(:i))

2

## Mutation

### Adding a tensor

In [24]:
tensor = Tensor(rand(2,2), [Index(:i), Index(:i2)])
addtensor!(tn, tensor)

SimpleTensorNetwork (#tensors=3, #inds=4)

### Renaming an index

In [25]:
replace_ind!(tn, Index(:i), Index(plug"1"))

SimpleTensorNetwork (#tensors=3, #inds=4)

In [26]:
replace!(tn, Index(:k) => Index(plug"2"))
replace!(tn, Index(:j) => Index(bond"1-2"))

SimpleTensorNetwork (#tensors=3, #inds=4)

In [27]:
inds(tn)

4-element Vector{Index}:
 Index{Bond{CartesianSite{1}, CartesianSite{1}}}((1,) <=> (2,))
 Index{Plug{CartesianSite{1}}}((2,))
 Index{Symbol}(:i2)
 Index{Plug{CartesianSite{1}}}((1,))

### Removing a tensor

In [28]:
rmtensor!(tn, tensor)

ArgumentError: ArgumentError: tensor not found

Index `:i` has been renamed and `Tensor`  is "immutable" (the `Tensor` object itself and the indices are immutable, but the array can be mutated if it allows so) so `tensor` has been replaced by another `Tensor`.

In [29]:
tensor = only(tensors(tn; contain=Index(:i2)))

2×2 Tensor{Float64, 2, Matrix{Float64}}:
 0.916427  0.292559
 0.942706  0.385372

In [30]:
rmtensor!(tn, tensor)

SimpleTensorNetwork (#tensors=2, #inds=4)

## Delegation

If you want to build another Tensor Network type on top of a type that implements the `TensorNetwork` interface (e.g. `GenericTensorNetwork`), TenetCore uses a "delegation" mechanism that automatically implements and calls the correct methods for you.

The only thing you need to do is to define `DelegatorTrait` for your type.

For more information, check out [DelegatorTraits.jl](https://github.com/bsc-quantic/DelegatorTraits.jl).

In [31]:
struct MyTensorNetwork <: TenetCore.AbstractTensorNetwork
    tn::SimpleTensorNetwork
end

MyTensorNetwork() = MyTensorNetwork(SimpleTensorNetwork())
TenetCore.DelegatorTrait(::TenetCore.TensorNetwork, tn::MyTensorNetwork) = TenetCore.DelegateTo{:tn}()

In [32]:
my_tn = MyTensorNetwork()

MyTensorNetwork (#tensors=0, #inds=0)

In [33]:
tensor = Tensor(rand(2,2), [Index(:i), Index(:j)])
addtensor!(my_tn, tensor)
tensors(my_tn)

1-element Vector{Tensor}:
 [0.9668505321162132 0.10147395859448316; 0.5473016117549419 0.18035314553882353]

In [34]:
replace_ind!(my_tn, Index(:i), Index(:new_i))

MyTensorNetwork (#tensors=1, #inds=2)

In [35]:
inds(my_tn)

2-element Vector{Index}:
 Index{Symbol}(:new_i)
 Index{Symbol}(:j)